# Neural Network Workshop

### First Application - Simple Feedforward Neural Network

This notebook was inspired from blog: https://victorzhou.com/blog/intro-to-neural-networks/

#### Importing Libraries


The NumPy library which is used for working with vectors/arrays. It handles operations much faster than traditional Python lists.

In [1]:
import numpy as np

#### Activation Function

In [2]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

#### Neuron Class

In [3]:
class Neuron:
  def __init__(self, weights, bias):
    self.weights = weights
    self.bias = bias

  def feedforward(self, inputs):
    total = np.dot(self.weights, inputs) + self.bias
    return sigmoid(total)

#### A single neuron

Our weights are [0,1] and our bias for the neuron is 4

In [4]:
weights = np.array([0, 1])
bias = 4                   
n = Neuron(weights, bias)

Using [2,3] as the inputs from two previous neurons. We apply our understanding of weights, biases, and the activation function (sigmoid) for a neuron to see that the output is...
![Calculations](img/perceptron.png)

In [5]:
x = np.array([2, 3])       
print(n.feedforward(x))    

0.9990889488055994


#### Neural Network Class

In [6]:
class FirstNeuralNetwork:
    def __init__(self):
        weights = np.array([0, 1])
        bias = 0
        self.h1 = Neuron(weights, bias)
        self.h2 = Neuron(weights, bias)
        self.o1 = Neuron(weights, bias)

    def feedforward(self, x):
        out_h1 = self.h1.feedforward(x)
        out_h2 = self.h2.feedforward(x)
        out_o1 = self.o1.feedforward(np.array([out_h1, out_h2]))
        return out_o1
        

Our feedforward neural network is ![Neural Network](img/network.png)

The calculations for our neural shows that with weights [0,1] and a bias of 0 for each hidden layer neuron and the output neuron yields...
![Calculations](img/feedforward.png)

In [7]:
network = FirstNeuralNetwork()
x = np.array([2, 3])
print(network.feedforward(x)) 

0.7216325609518421


### Second Application - Classification ANN w/ Tensorflow

#### Importing the TensorFlow and Keras libraries.

TensorFlow is an open-source machine learning library for Python and Keras is an interface for creating neural networks. There is a video on using TensorFlow with neural networks on the Waterloo Data Science Club Youtube channel.

In [8]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import layers

#### Fashion-MNIST dataset.

We will be using the Fashion-MNIST dataset. The Fashion-MNIST dataset is comprised of 60,000 training images and 10,000 testing images. All of the images are labelled. Our goal is to be able to accurately classify images from the dataset. The load_data() function loads the training and testing images and labels into the appropriate NumPy arrays.

In [9]:
from tensorflow.keras.datasets import fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [10]:
print(type(x_train))
print(type(y_train))
print(type(x_test))
print(type(y_test))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


#### Labels for dataset

In [11]:
labels = ['T-shirt', 'Trousers', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Boot']

#### Plotting samples

In [12]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(5, 5))
for i in range(1,10):
    fig.add_subplot(3,3,i)
    plt.imshow(x_train[i], cmap=plt.get_cmap('gray'))
    plt.title(labels[y_train[i]])
fig.tight_layout()
plt.show()

<Figure size 500x500 with 9 Axes>

#### Normalizing images

In grayscale, pixel intensities vary from 0 (black) to 255 (white), and everything in between are shades of grey. Here is the results from printing the pixel intensities of an image in the dataset.

In [13]:
print(x_train[1])

[[  0   0   0   0   0   1   0   0   0   0  41 188 103  54  48  43  87 168
  133  16   0   0   0   0   0   0   0   0]
 [  0   0   0   1   0   0   0  49 136 219 216 228 236 255 255 255 255 217
  215 254 231 160  45   0   0   0   0   0]
 [  0   0   0   0   0  14 176 222 224 212 203 198 196 200 215 204 202 201
  201 201 209 218 224 164   0   0   0   0]
 [  0   0   0   0   0 188 219 200 198 202 198 199 199 201 196 198 198 200
  200 200 200 201 200 225  41   0   0   0]
 [  0   0   0   0  51 219 199 203 203 212 238 248 250 245 249 246 247 252
  248 235 207 203 203 222 140   0   0   0]
 [  0   0   0   0 116 226 206 204 207 204 101  75  47  73  48  50  45  51
   63 113 222 202 206 220 224   0   0   0]
 [  0   0   0   0 200 222 209 203 215 200   0  70  98   0 103  59  68  71
   49   0 219 206 214 210 250  38   0   0]
 [  0   0   0   0 247 218 212 210 215 214   0 254 243 139 255 174 251 255
  205   0 215 217 214 208 220  95   0   0]
 [  0   0   0  45 226 214 214 215 224 205   0  42  35  60  16  1

We change the type of the pixel intensities in the images from integer to float so that we can normalize. We normalize pixel intensities to make calculating the gradient easier and so that the convergence speed is improved.

In [14]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

#### Reshaping data

We are reshaping the data so that the images are flattened from 2-dimension to 1-dimensions. The original images in the Fashion-MNIST dataset are 28 by 28 pixels so a 1-dimensional array will have 784 elements corresponding to every pixel.

In [15]:
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

#### One-hot encoding

![alt text](https://www.researchgate.net/profile/Jose_Padarian/publication/334507958/figure/fig1/AS:781537293377537@1563344419499/Example-of-two-encodings-of-the-phrase-red-sticky-clay-numerical-encoding-and-one-hot.ppm)
At the moment, the labels for all the images range from 0 to 9 and they refer to the element in the labels list. We one-hot encode them to create vectors which correspond to the output layer of the neural network when the model predicts a particular label.

In [16]:
from keras.utils import to_categorical

y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

Using TensorFlow backend.


#### Building the model

In [17]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation


model = Sequential()
model.add(Dense(784, activation='sigmoid', input_dim=784))
model.add(Dense(30, activation='sigmoid'))
model.add(Dense(10, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 784)               615440    
_________________________________________________________________
dense_2 (Dense)              (None, 30)                23550     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                310       
Total params: 639,300
Trainable params: 639,300
Non-trainable params: 0
_________________________________________________________________


#### Compiling model

Compiling the model means we are how we will train the model. We are using the categorical crossentropy loss function. We are using the stochastic gradient descent for our optimizer. The metric that we will be using is accuracy.

In [18]:
model.compile(loss="categorical_crossentropy",
              optimizer="sgd",
              metrics = ['accuracy'])

#### Training the model

In [19]:
model.fit(x_train, y_train, batch_size=50, epochs=10)

Epoch 1/10
60000/60000 [==============================] - 3s 45us/step - loss: 2.0232 - accuracy: 0.4704
Epoch 2/10
60000/60000 [==============================] - 3s 49us/step - loss: 1.4606 - accuracy: 0.6176
Epoch 3/10
60000/60000 [==============================] - 3s 48us/step - loss: 1.1653 - accuracy: 0.6493
Epoch 4/10
60000/60000 [==============================] - 3s 43us/step - loss: 0.9956 - accuracy: 0.6851
Epoch 5/10
60000/60000 [==============================] - 3s 43us/step - loss: 0.8847 - accuracy: 0.7103
Epoch 6/10
60000/60000 [==============================] - 3s 43us/step - loss: 0.8089 - accuracy: 0.7278
Epoch 7/10
60000/60000 [==============================] - 3s 42us/step - loss: 0.7551 - accuracy: 0.7374
Epoch 8/10
60000/60000 [==============================] - 3s 42us/step - loss: 0.7152 - accuracy: 0.7441
Epoch 9/10
60000/60000 [==============================] - 3s 46us/step - loss: 0.6842 - accuracy: 0.7532
Epoch 10/10
60000/60000 [==============================

#### Evaluating model

In [20]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(print('Test accuracy:', round(test_acc,4)))

10000/10000 [==============================] - 0s 28us/step
Test accuracy: 0.7596
None
